In [1]:
import pyodbc
import os
from datetime import date, datetime, timedelta
import pandas as pd
import win32com.client as win32
from databases import Bawm, Crm,PosicaoDm1,PosicaoDm1Pickle,Boletador
from funcoes_datas import FuncoesDatas
import os
import numpy as np
import xlwings as xw
from dateutil.relativedelta import relativedelta
import unicodedata

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
fdt = FuncoesDatas()
crm = Crm()
produtos = crm.rating_produtos()
posicao = PosicaoDm1()
peacke = PosicaoDm1Pickle()
bawm = Bawm()
bol = Boletador()

In [3]:
def remove_acentos(texto):
     return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')

In [4]:
#retornando dados cadastrais das carteiras
dados_po_cadastro = bawm.po_cadastro(apenas_fundos=False)[['GuidContaCRM','NomeContaCRM']]
dados_crm= crm.account()[['accountid','createdon']].rename(columns={'accountid':'GuidContaCRM'})
dados_crm['GuidContaCRM'] = dados_crm['GuidContaCRM'].str.lower()

In [5]:
with open ('df_fundos_2024-11-11.pickle','rb') as f:
    pickle.load(f,fix_imports=True,encoding='latin1')

NameError: name 'pickle' is not defined

In [ ]:
#retorna o ultimo dia e penultimo dia
data = fdt.hoje()
ultimo_extrato = data - relativedelta(days=data.day -1)
ultimo_extrato = fdt.workday(ultimo_extrato,-1)
data_fim = datetime.date(ultimo_extrato)

penultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
data_ini = datetime.date(penultimo_extrato)
penultimo_extrato = datetime.date(fdt.workday(penultimo_extrato,-1))
penultimo_extrato = '_' + str(penultimo_extrato.year) + '-' + '{:02d}'.format(penultimo_extrato.month)  + '-' + str(penultimo_extrato.day)

#Transforma a data para ler o peakle
caminho1 = f'O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_fundos{penultimo_extrato}.pickle'
caminho2 = f'O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_tit{penultimo_extrato}.pickle'

# #le o pickle
carteira_fundos = pd.read_pickle(caminho1)
carteira_sc = pd.read_pickle(caminho2)
carteira_penultimo_mes = pd.concat([carteira_fundos,carteira_sc])

#Trazer a carteira do ultimo mes

carteira_ultimo_mes = posicao.posicao_total_depara(ultimo_extrato)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_ultimo_mes=carteira_ultimo_mes.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_ultimo_mes['GuidProduto']= carteira_ultimo_mes['GuidProduto'].str.lower()
#Retirar da carteira Caixa e Compromissada
carteira_ultimo_mes = carteira_ultimo_mes[(carteira_ultimo_mes['TipoProduto']!='CAIXA')&(carteira_ultimo_mes['TipoProduto']!='COMP')]

#Trazer a carteira do penultimo mês

carteira_penultimo_mes = carteira_penultimo_mes[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_penultimo_mes =carteira_penultimo_mes .sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_penultimo_mes ['GuidProduto']= carteira_penultimo_mes ['GuidProduto'].str.lower()
#Retirar da carteira Caixa e Compromissada
carteira_penultimo_mes  = carteira_penultimo_mes [(carteira_penultimo_mes ['TipoProduto']!='CAIXA')&(carteira_penultimo_mes ['TipoProduto']!='COMP')]

#retornar os clientes novos
clientes_ultimo_mes = carteira_ultimo_mes[['NomeContaCRM','TipoProduto']]
clientes_penultimo_mes = carteira_penultimo_mes[['NomeContaCRM','TipoProduto']]
novos = pd.merge(left=clientes_ultimo_mes,right=clientes_penultimo_mes,on='NomeContaCRM',how='left')
clientes_novos = novos[novos['TipoProduto_y'].isnull()].drop_duplicates()
clientes_novos = clientes_novos['NomeContaCRM'].drop_duplicates()

#clientes_novos
clientes_novos = pd.merge(left=clientes_novos,right=dados_po_cadastro,on='NomeContaCRM',how='left')
clientes_novos = pd.merge(left=clientes_novos,right=dados_crm,on='GuidContaCRM',how='left')
clientes_novos = clientes_novos.drop_duplicates().sort_values(by=['createdon'],ascending=False)



In [ ]:
#Ativos aprovados por credito
#Produtos aprovados pela casa

pasta = 'O:/SAO/CICH_All/Real Estate/1. Users/Guilherme Liberatore/2. Real Estate/3. Monitoramento/PDFs'
ativos_imob = os.listdir(pasta)
ativos_imob = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_imob]

pasta = 'O:/SAO/CICH_All/Renda Variável/Monitoramento/Monitoramentos/2024.1T'
ativos_rv = os.listdir(pasta)
ativos_rv  = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_rv]

corporativos =  pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')
corporativos  = corporativos[(corporativos['Status']=='Recomendado')]
corporativos['CETIP'] = corporativos['CETIP'].str.lower()
corporativos = list(corporativos['CETIP'])

estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')
estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')[['IdPAS']].astype('str')
estruturados = estruturados[estruturados['IdPAS'].notnull()].dropna()
estruturados = pd.merge(left=estruturados,right=produtos,on='IdPAS',how='left')

estruturados['CETIP'] = estruturados['CETIP'].str.lower()
estruturados['ISIN'] = estruturados['ISIN'].str.lower()
estruturado_cetip = list(estruturados['CETIP'])
estruturado_isin= list(estruturados['ISIN'])

#Gestores aprovados
gestores = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Gestores')[['Gestora']]
gestores =list(gestores['Gestora'].str.lower())

#Códigos dos ativos aprovados

aprovados = ativos_rv + ativos_imob+corporativos+estruturado_cetip+estruturado_isin
aprovados = [aprovado for aprovado in aprovados if str(aprovado) != 'nan']
aprovados = [aprovado for aprovado in aprovados if aprovado != np.nan]
aprovados = [aprovado for aprovado in aprovados if aprovado != None]

produtos_aprovados = pd.DataFrame(aprovados)

In [ ]:
#Trazer todos os produtos e suas informações para complementar as decisões do comite

produtos = crm.rating_produtos()
produtos['GuidProduto']=produtos['GuidProduto'].str.lower()
produtos['CETIP']=produtos['CETIP'].str.lower()
produtos['ISIN']=produtos['ISIN'].str.lower()

#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
n_produtos_comite = len(produtos_comite)
produtos_comite_maio = len(produtos_comite[produtos_comite['Data Comitê']>='2024-05-01'])
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Unifica as bases
Ativos_comite = pd.concat([cnpj,isin,cetip])

#Trazer todos os clientes que passaram pelo comite

#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_ultimo_mes['NomeContaCRM'].drop_duplicates().to_list()
substr = Ativos_comite['Cliente'].drop_duplicates().to_list()
#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]
clientes_comite = Filter(string, substr)
clientes_comite_analise = clientes_comite



In [ ]:
#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_ultimo_mes['NomeContaCRM'].drop_duplicates().to_list()
substr = Ativos_comite['Cliente'].drop_duplicates().to_list()
#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]
clientes_comite = Filter(string, substr)


In [ ]:
#Traz os ativos novos
deliberacoes = Ativos_comite[['GuidProduto','Data Comitê','Deliberação','Analisado Pelo Comitê']]
ativos_ultimo_mes = carteira_ultimo_mes[['GuidProduto','NomeProduto','TipoProduto']]
ativos_penultimo_mes = carteira_penultimo_mes[['GuidProduto','NomeProduto','TipoProduto']]
ativos_ultimo_mes = ativos_ultimo_mes.drop_duplicates()
ativos_penultimo_mes = ativos_penultimo_mes.drop_duplicates()
ativos_novos = pd.merge(left=ativos_ultimo_mes,right=ativos_penultimo_mes,on='GuidProduto',how='left')
ativos_novos=ativos_novos[ativos_novos['TipoProduto_y'].isnull()]
ativos_novos = ativos_novos[['GuidProduto','NomeProduto_x','TipoProduto_x']].rename(columns={'NomeProduto_x':'NomeProduto','TipoProduto_x':'TipoProduto'}).reset_index(drop=True)
ativos_novos=pd.merge(left=ativos_novos, right=produtos, on='GuidProduto',how='left')
ativos_novos=pd.merge(left=ativos_novos, right=deliberacoes, on='GuidProduto',how='left')

#Retirando todos os cod aprovados CRM
ativos_novos_cetip=list(ativos_novos['CETIP'].str.lower())
ativos_novos_cetip = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_cetip ]
ativos_novos['CETIP_verificar'] = ativos_novos_cetip

# #Retirando todos os cod aprovados CRM
ativos_novos_ticker=list(ativos_novos['IdPAS'].str.lower())
ativos_novos_ticker = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_ticker]
ativos_novos['Ticker_verificar'] = ativos_novos_ticker

#Bancos emissores aprovados
emissores_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
emissores_aprovados = list(emissores_aprovados['Banco'])
emissores = ativos_novos['emissor']
emissores_aprovados = ['Recomendado' if emissor in emissores_aprovados else 'Reprovado' for emissor in emissores]
emissores_aprovados = [emissor for emissor in emissores_aprovados if str(emissor) != 'nan']
ativos_novos['Emissor_Aprovado'] = emissores_aprovados

In [ ]:
ativos_novos['Recomendado_Investimentos'] = ativos_novos.apply(lambda x: list([x['CETIP_verificar'],x['Ticker_verificar'],x['Emissor_Aprovado']]),axis=1)
comite_list = list(ativos_novos['Recomendado_Investimentos'])
aprovados = []
for x in comite_list:
    if ('Monitorado' in x) or ('Recomendado' in x):
        decisao = 'Recomendado'
    else:
        decisao = 'Reprovado'
    aprovados.append(decisao)   
ativos_novos['Recomendado_Investimentos'] = aprovados
ativos_novos['Rating'].loc[ativos_novos['Recomendado_Investimentos']=='Recomendado'] = 'Recomendado'
ativos_novos = ativos_novos.dropna(subset=['GuidProduto'])

In [ ]:
#Geração de estatistica de ativos (Quantificando o quanto de ativos novos e o seu rating)

ativos_novos = ativos_novos.drop_duplicates(subset=['GuidProduto'])
n_ativos_novos = len(ativos_novos['GuidProduto'].drop_duplicates())
n_ativos_reprovados = len(ativos_novos[(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')])
n_ativos_monitorados = len(ativos_novos[ativos_novos['Rating']=='Monitorado'])

n_ativos_comite = len(ativos_novos[ativos_novos['Deliberação'].notnull()])
n_ativos_comite_reprovados = len(ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')])
n_ativos_comite_monitorados = len(ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']=='Monitorado')])

#Retorna os produtos

ativos_reprovados = ativos_novos[(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')]
ativos_monitorados = ativos_novos[ativos_novos['Rating']=='Monitorado']

ativos_comite = ativos_novos[ativos_novos['Deliberação'].notnull()]
ativos_comite_reprovados = ativos_comite[ativos_comite['Analisado Pelo Comitê']=='Comitê']
ativos_comite_reprovados = ativos_comite_reprovados[ativos_comite_reprovados['Deliberação'].str.contains('Zerar')]
#ativos_comite_reprovados = ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')]
ativos_comite_monitorados = ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']=='Monitorado')]

In [ ]:
string = carteira_ultimo_mes['NomeContaCRM'].drop_duplicates().to_list()
substr = Ativos_comite['Cliente'].drop_duplicates().to_list()
#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)
def Filter(string, substr):
    return [ str for str in substr if
             any(sub in str for sub in string)]
clientes_entraram = Filter(string, substr)

In [ ]:
#Retorna apenas os clientes do comite que entraram na D1

#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_ultimo_mes['NomeContaCRM'].drop_duplicates().to_list()
substr = Ativos_comite['Cliente'].drop_duplicates().to_list()
#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)
def Filter(string, substr):
    return [str for str in substr if
             any(sub in str for sub in string)]
clientes_entraram = Filter(string, substr)


clientes_entraram = len(clientes_entraram)
clientes_analisados = len(substr)
print(str(clientes_analisados) + ' ' + str(clientes_entraram))

In [ ]:
def Filter(string, substr):
    return [str for str in substr if
             any(sub in str for sub in string)]
clientes_entraram = Filter(string, substr)

In [ ]:
#Analise das carteiras do comite - Analise Descritiva

import os

#Carteira dos clientes
carteiras_analise = carteira_ultimo_mes[carteira_ultimo_mes['NomeContaCRM'].isin(clientes_comite_analise)]

#Por segmento
cadastro_analise = bawm.po_cadastro()

#Comite de exceção
comite_analise = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Análises Iniciais - Ativos/Planilha Comitê de Exceção - BrunoB_.xlsx')

#Verificar a linha que começa o dataframe para coleta de dados
colunas = comite_analise.columns
cotistas = []
transferencia = []
for x in colunas:
    comite_analise[x] = comite_analise[x].astype('str')
    try:
            linha = list(comite_analise[x]).index('Transferência')
    except:    
            pass


In [ ]:
#Puxa toda a analise inicial de ativos da pasta

import time

pasta = 'O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Análises Iniciais - Ativos'
arquivos = os.listdir(pasta)
for arq in arquivos:  
    data = os.path.getmtime(pasta + '/'+arq)
    data = time.ctime(data)
    data = pd.to_datetime(data)
    arquivo = pasta + '/' + arq
    arquivo = arquivo.replace('~$','')       
    dia = datetime(2023,12,1)
    if (arquivo.endswith('.xlsx')) and data > dia :
        try:
            arquivo = pd.read_excel(pasta + '/' + arq)
            colunas = arquivo.columns
            for x in colunas:
                comite_analise[x] = comite_analise[x].astype('str')
            try:
                    linha = list(comite_analise[x]).index('Transferência')
            except:    
                pass

        except:
            pass

In [ ]:
#Verificar o quanto de ativos que passaram no comite foram vendidos

clientes_comite_sql = ",".join([f"'{item}'" for item in clientes_comite])
posicao_dia= posicao.posicao_range_comite_excecao(data_ini,data_fim,clientes_comite_sql)
boletas_feitas_periodo = boletas = bol.boletas_range_datas(data_ini=data_ini, data_fim=data_fim, exportado=True, deletado=False)
boletas_feitas_periodo = boletas_feitas_periodo[boletas_feitas_periodo['TipoMov']=='V']
boletas_feitas_periodo = boletas_feitas_periodo.rename(columns={'AtivoGuid':'GuidProduto','ContaCRM':'NomeContaCRM'})
boletas_feitas_periodo = boletas_feitas_periodo[['GuidProduto','DataMov','NomeContaCRM','Financeiro',]]
posicao_dia = posicao_dia[['NomeContaCRM','GuidProduto', 'TipoProduto','NomeProduto','FinanceiroFinal']]
posicao_dia = posicao_dia.groupby(['NomeContaCRM','GuidProduto', 'TipoProduto','NomeProduto']).sum()
posicao_dia = posicao_dia.reset_index().drop_duplicates()
posicao_dia['GuidProduto']=posicao_dia['GuidProduto'].str.lower()
ativos_deliberacao = Ativos_comite[['GuidProduto','Data Comitê','Deliberação','Analisado Pelo Comitê']]

#Ativos vendidos esse mês (dos clientes do comitê)
verificar = pd.merge(left=posicao_dia, right=boletas_feitas_periodo, on=['NomeContaCRM','GuidProduto'],how='left')
verificar = pd.merge(left=verificar, right=ativos_deliberacao, on='GuidProduto',how='left')
verificar_vendas = verificar[(verificar['Financeiro'].notnull())&(verificar['Deliberação'].str.contains('Zerar'))]
verificar_vendas = verificar_vendas[verificar_vendas['Analisado Pelo Comitê']=='Comitê']


In [ ]:
#O que entrou esse mes que precisa ser zerado
zeragem = len(ativos_deliberacao[ativos_deliberacao['Deliberação'].str.contains('Zerar')])
print(f'entraram {len(ativos_comite_reprovados)} nesse mês dos {zeragem} que precisam ser zerados no total')

In [ ]:
#Vendas desde o inicio do ano
data_ini = datetime(2023,10,1)
ativos_deliberacao['Deliberação'] = ativos_deliberacao['Deliberação'].astype('str')
vendas = ativos_deliberacao[ativos_deliberacao['Deliberação'].str.contains('Zerar')][['GuidProduto']].dropna().drop_duplicates()
boletas_feitas_ano = boletas = bol.boletas_range_datas(data_ini=data_ini, data_fim=data_fim, exportado=True, deletado=False)
boletas_feitas_ano = boletas_feitas_ano.reset_index()
boletas_feitas_ano = boletas_feitas_ano.loc[boletas_feitas_ano['ContaCRM'].isin(clientes_comite)]
boletas_feitas_ano = boletas_feitas_ano[boletas_feitas_ano['TipoMov']=='V']
vendas = list(vendas['GuidProduto'])
boletas_feitas_ano = boletas_feitas_ano.loc[boletas_feitas_ano['AtivoGuid'].isin(vendas)]
boletas_feitas_ano.to_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Comite Exceção Mensal/vendas_2023.xlsx')

In [ ]:
#Verificar quantos clientes do comitê entraram na carteira

carteira_clientes_comite = posicao.posicao_total_depara(fdt.hoje())
#clientes_comite_d1 = list(clientes_carteira['NomeContaCRM'].drop_duplicates())

#Quantos dos clientes novos entraram via comite
def Filter(string, substr):
    return [ str for str in substr if
             any(sub in str for sub in clientes_comite)]
clientes_d1_comite = Filter(clientes_comite, substr)
print(f'Dos {str(len(substr))} analisados via comitê de exceção, {str(len(clientes_d1_comite))} entraram para nossa gestão ')


In [ ]:
#Verificar ativos do comitê que estão na carteira
carteira_clientes_comite = carteira_clientes_comite.loc[carteira_clientes_comite['NomeContaCRM'].isin(clientes_comite)][['NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']].drop_duplicates().sort_values(by='NomeContaCRM').reset_index(drop=True)
carteira_clientes_comite = pd.merge(left = carteira_clientes_comite, right=produtos, on='GuidProduto',how='left')

In [ ]:
#Análise do Comite - Insights

#Verificar a lista dos nomes que estão na D0 e como estão no comite

cadastro = posicao.po_cadastro(apenas_fundos=False)
cadastro = cadastro[['NomeContaCRM','Segmento','Officer']]

def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]
clientes_comite = Filter(string, substr)

#Verificação dos ativos já deliberados (verificar se o banco é aprovado ou ativo está na lista de crédito)

#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite['Deliberação'] = produtos_comite['Deliberação'].astype('str')
produtos_comite = produtos_comite[produtos_comite['Deliberação'].str.contains('Zerar')]
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite['Cliente'] =produtos_comite['Cliente'].apply(lambda x: remove_acentos(x) )

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Puxar os Pls das carteiras e fundos
pls = bawm.patrimoniodata()
pls = pls[['NomeContaCRM','PLInicial']].rename({'PLInicial':'PL'})

#Unifica as bases
ativos_deliberados = pd.concat([cnpj,isin,cetip])
ativos_deliberados['Ativo_x'] = ativos_deliberados['Ativo_x'].str.lower()

#Puxar o Guid de todos os produtos que tiveram deliberação
guids_produtos = list(ativos_deliberados['GuidProduto'].drop_duplicates())

#Puxar todos os bancarios dos clientes
ativos_bancos = ['CDB','LCI','LCA','LIG','LFS','LC','DPGE','LF']

#Retornar as carteiras que avaliamos via comite
hoje = fdt.hoje()
carteira_atual = posicao.posicao_total_depara(hoje)
carteira_atual = carteira_atual.loc[carteira_atual['NomeContaCRM'].isin(clientes_comite_analise)]
carteira_atual_n_bancarios = carteira_atual.loc[carteira_atual['GuidProduto'].isin(guids_produtos)]
carteira_atual_n_bancarios =carteira_atual_n_bancarios.loc[~carteira_atual_n_bancarios['TipoProduto'].isin(ativos_bancos)]
carteira_atual_n_bancarios['bancario'] = np.nan
carteira_atual_bancarios = carteira_atual.loc[carteira_atual['TipoProduto'].isin(ativos_bancos)]
carteira_atual_bancarios['bancario'] = 'sim'

#Bancos emissores aprovados (alterando o ranking do CRM para nosso ranking)
emissores_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
emissores_aprovados = list(emissores_aprovados['Banco'])
emissores = carteira_atual_bancarios['NomeEmissor']
emissores_aprovados = ['Recomendado' if emissor in emissores_aprovados else 'Reprovado' for emissor in emissores]
emissores_aprovados = [emissor for emissor in emissores_aprovados if str(emissor) != 'nan']
carteira_atual_bancarios['RatingGPS'] = emissores_aprovados

#unificando as carteiras
carteira_comite_analise = pd.concat([carteira_atual_n_bancarios,carteira_atual_bancarios])

#Unindo com as informações de PL
carteira_comite_analise = pd.merge(left=carteira_comite_analise,right=pls, on='NomeContaCRM',how='left')

#Total do financeiro de clientes que passaram via comite
pl = carteira_comite_analise['FinanceiroFinal'].sum()
qtd_ativos = carteira_comite_analise['NomeContaCRM'].count()

#Total que entrou via comite por segmento
segmento =  pd.merge(left=carteira_comite_analise,right=cadastro, on='NomeContaCRM',how='left')
segmento = segmento.drop_duplicates()


#Deixando letra minuscula para agrupar
segmento['RatingGPS'] = segmento['RatingGPS'].str.lower()

#Rating do ativo por segmento
segmento_financeiro_rating = segmento.groupby(['Segmento','RatingGPS']).sum().reset_index()[['Segmento','RatingGPS','FinanceiroFinal']]
segmento_qtd_ativos_rating = segmento.groupby(['Segmento','RatingGPS']).count().reset_index()[['Segmento','RatingGPS','FinanceiroFinal']].rename(columns={'FinanceiroFinal':'Quantidade_Ativos'})

#Retornando os dados consolidados e mostrando o percentual financeiro e quantidade referente a rating por segumento.
analise_1 = pd.concat([segmento_financeiro_rating,segmento_qtd_ativos_rating],axis=1)
analise_1 = analise_1.loc[:,~analise_1.columns.duplicated()]
analise_1['%Financeiro'] = analise_1['FinanceiroFinal']/pl*100
analise_1['%Quantidade_Ativos'] = analise_1['Quantidade_Ativos']/qtd_ativos*100
#Mostrando os dados de outra forma
analise_2 = pd.pivot_table(data =analise_1,index='Segmento',values=['Quantidade_Ativos','FinanceiroFinal'],columns=['RatingGPS']).reset_index()

#Mostrando o que entrou na nossa gestão vindo do comitê por segmento
#Rating do ativo por segmento
segmento_financeiro_total = segmento.groupby(['Segmento']).sum().reset_index()[['Segmento','FinanceiroFinal']]
segmento_qtd_ativos_total = segmento.groupby(['Segmento']).count().reset_index()[['Segmento','FinanceiroFinal']].rename(columns={'FinanceiroFinal':'Quantidade_Ativos'})
analise_3 = pd.concat([segmento_financeiro_total,segmento_qtd_ativos_total],axis=1)
analise_3 = analise_3.loc[:,~analise_3.columns.duplicated()]
analise_3['%Financeiro'] = analise_3['FinanceiroFinal']/pl*100
analise_3['%Quantidade_Ativos'] = analise_3['Quantidade_Ativos']/qtd_ativos*100

#Analise Segmento por tipo de transferencia
analise_deliberacao = Ativos_comite[['Cliente','GuidProduto','Deliberação']]
trasferencia = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Levantamento Transferencia.xlsx')
analise_deliberacao = pd.merge(left = analise_deliberacao,right = trasferencia, on ='Cliente',how='left')
analise_deliberacao = analise_deliberacao.dropna(subset=['Segmento'])
analise_deliberacao['Segmento'] = analise_deliberacao['Segmento'].str.lower()
analise_deliberacao = analise_deliberacao.groupby(['Deliberação','Segmento','Tranferencia ']).count().reset_index()[['Tranferencia ','Deliberação','Segmento','Cliente']].sort_values(by=['Tranferencia ','Deliberação'])
total = analise_deliberacao['Cliente'].sum()
analise_deliberacao['%Cliente'] =  analise_deliberacao['Cliente'] / total *100

analise_4 = pd.pivot_table(data =analise_deliberacao,index='Segmento',values=['Cliente'],columns=['Deliberação','Tranferencia ']).reset_index()

#Analise apenas das deliberações zerar
analise_5 = analise_deliberacao[analise_deliberacao['Deliberação'].str.contains('Zerar')]
total = analise_5['Cliente'].sum()
analise_5['%Cliente'] = analise_5['Cliente'] / total *100

#Analise por tipo de ativo que entrou para nossa gestão e deliberação
ativos_analise = ativos_deliberados[['GuidProduto','Deliberação']].drop_duplicates(subset=['GuidProduto'])
ativos_analise = pd.merge(left=carteira_comite_analise,right=ativos_analise,on='GuidProduto',how='left')
ativos_analise =  pd.merge(left=ativos_analise,right=cadastro, on='NomeContaCRM',how='left')
ativos_analise = ativos_analise.drop_duplicates()
#Rating do ativo por segmento
ativos_analise_fin_deliberacao = ativos_analise.groupby(['Segmento','Deliberação']).sum().reset_index()[['Segmento','Deliberação','FinanceiroFinal']]
ativos_analise_qtd_deliberacao = ativos_analise.groupby(['Segmento','Deliberação']).count().reset_index()[['Segmento','Deliberação','FinanceiroFinal']].rename(columns={'FinanceiroFinal':'Quantidade_Ativos'})

#Retornando os dados consolidados e mostrando o percentual financeiro e quantidade referente a rating por segumento.
analise_6 = pd.concat([ativos_analise_fin_deliberacao,ativos_analise_qtd_deliberacao],axis=1)
analise_6 = analise_6.loc[:,~analise_6.columns.duplicated()]
analise_6 = analise_6.sort_values(by=['Deliberação','Quantidade_Ativos','FinanceiroFinal'],ascending=False)
analise_6['%Financeiro'] = analise_6['FinanceiroFinal'] / analise_6['FinanceiroFinal'].sum()*100


In [ ]:
#Analise de custo dos ativos que entram via comite de exceção

#Puxando as informações dos ativos que conseguimos preços via ANBIMA e o preço por tipo de produto que vem do pop
custo_prod_anb = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Comite Exceção Mensal/Produtos Preco Anbima.xlsx')
custo_prod_pop = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Comite Exceção Mensal/Produtos Preco Anbima.xlsx',sheet_name='POP')

#Preços da PLanilha vida popo e unindo isso a carteira de ativos
analise_custo = pd.merge(left = carteira_comite_analise, right=custo_prod_anb,on='CodigoCetip',how='left').drop_duplicates().drop(columns={'Tipo Produto'})
analise_custo = pd.merge(left = analise_custo, right=custo_prod_pop,on='TipoProduto',how='left').drop_duplicates().drop(columns={'M'})

#Trazendo os dados das carteiras (segmento)
analise_custo =  pd.merge(left=analise_custo,right=cadastro, on='NomeContaCRM',how='left')
analise_custo = analise_custo.drop_duplicates()

#Retornando o preço por ativo
analise_custo['Y'].loc[analise_custo['Preço']=='Anbima'] =0 
analise_custo['Preço'] = analise_custo['Y']
analise_custo['Preço'] = analise_custo['Preço'].fillna(0)
analise_custo = analise_custo.drop(columns={'Y'})

#Função de preços cobrados por gestão da carteira por faixa de PL

def fee(pl):
    if pl <= 15000000:
        fee = 0.75
    elif (pl > 15000000) and (pl <= 30000000):
        fee = 0.65
    elif (pl > 30000000) and (pl <= 50000000):
        fee = 0.55
    elif (pl > 50000000) and (pl <= 80000000):
        fee = 0.50
    elif (pl > 80000000) and (pl <= 100000000):
        fee = 0.45
    else:
        fee = 0.35
        
    return fee    

In [ ]:
#Custo de Precificacao da gestão por tipo de PL
analise_custo['Segmento'].drop_duplicates()
analise_custo = analise_custo[['NomeContaCRM','QuantidadeFinal','FinanceiroFinal','PLInicial','Segmento','Preço']]
analise_custo['Custo_JBB'] = analise_custo['PLInicial'].apply(lambda x: fees(x)/100)
                                                            

In [ ]:
#Analise do custo por cliente
analise_receita = analise_custo[['NomeContaCRM','PLInicial']].drop_duplicates()
analise_receita['Receita'] = analise_custo['Custo_JBB'] * analise_custo['PLInicial']
analise_custo = analise_custo.groupby(['NomeContaCRM','Segmento','Custo_JBB']).sum()[['Preço']].reset_index()

analise_custo = pd.merge(left = analise_receita,right=analise_custo,on='NomeContaCRM',how='left')
analise_custo['%Receita'] = analise_custo['Preço']/analise_custo['Receita']*100
analise_custo.to_excel('analise_custo_conta.xlsx')

#Analise custo por segmento
analise_custo = analise_custo.groupby(['Segmento']).sum().reset_index()[['Segmento','PLInicial','Receita','Preço']]
analise_custo['%Receita'] = analise_custo['Preço']/analise_custo['Receita']*100
analise_custo.to_excel('analise_custo_segmento.xlsx')

In [ ]:
#Gerando excel para e-mail:
with pd.ExcelWriter('C:/Temp/Foundation/Comite/Analise.xlsx') as writer:  
    analise_1.to_excel(writer, sheet_name='analise_1')
    analise_2.to_excel(writer, sheet_name='analise_2')
    analise_3.to_excel(writer, sheet_name='analise_3')
    analise_4.to_excel(writer, sheet_name='analise_4')
    analise_5.to_excel(writer, sheet_name='analise_5')
    analise_6.to_excel(writer, sheet_name='analise_6')
    analise_custo.to_excel('analise_custo.xlsx')
    analise_custo_conta.to_excel('analise_custo.xlsx')

In [ ]:
# #Retirando todos os cod aprovados CRM
# ativos_novos_cetip=list(carteira_clientes_comite['CETIP'].str.lower())
# ativos_novos_cetip = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_cetip ]
# carteira_clientes_comite['CETIP_verificar'] = ativos_novos_cetip

# # #Retirando todos os cod aprovados CRM
# ativos_novos_ticker=list(carteira_clientes_comite['IdPAS'].str.lower())
# ativos_novos_ticker = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_ticker]
# carteira_clientes_comite['Ticker_verificar'] = ativos_novos_ticker

# #Bancos emissores aprovados
# emissores_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
# emissores_aprovados = list(emissores_aprovados['Banco'])
# emissores = carteira_clientes_comite['emissor']
# emissores_aprovados = ['Recomendado' if emissor in emissores_aprovados else 'Reprovado' for emissor in emissores]
# emissores_aprovados = [emissor for emissor in emissores_aprovados if str(emissor) != 'nan']
# carteira_clientes_comite['Emissor_Aprovado'] = emissores_aprovados

# carteira_clientes_comite['Recomendado_Investimentos'] = carteira_clientes_comite.apply(lambda x: list([x['CETIP_verificar'],x['Ticker_verificar'],x['Emissor_Aprovado']]),axis=1)
# comite_list = list(carteira_clientes_comite['Recomendado_Investimentos'])
# aprovados = []
# for x in comite_list:
#     if ('Monitorado' in x) or ('Recomendado' in x):
#         decisao = 'Recomendado'
#     else:
#         decisao = 'Reprovado'
#     aprovados.append(decisao)   
# carteira_clientes_comite['Recomendado_Investimentos'] = aprovados
# carteira_clientes_comite['Rating'].loc[ativos_novos['Recomendado_Investimentos']=='Recomendado'] = 'Recomendado'
# ativos_novos = ativos_novos.dropna(subset=['GuidProduto'])